**Alexnet**

In [6]:
import pandas as pd
import numpy as np
import glob, os
import os
from tensorflow import keras
#import tensor as tf
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Convolution2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.initializers import glorot_uniform
import numpy as np
np.random.seed(1000)

In [7]:
PATH = "../input/rsna-str-pulmonary-embolism-detection/"
for item in os.listdir(PATH):
    path = os.path.join(PATH, item) 
    if os.path.isfile(PATH):
        print(PATH)
train_df = pd.read_csv(PATH + "train.csv")
test_df = pd.read_csv(PATH + "test.csv")

TRAIN_PATH = PATH + "train/"
TEST_PATH = PATH + "test/"
sub = pd.read_csv(PATH + "sample_submission.csv")
train_image_file_paths = glob.glob(TRAIN_PATH + '/*/*/*.dcm')
test_image_file_paths = glob.glob(TEST_PATH + '/*/*/*.dcm')

print(f'Train dataframe shape  :{train_df.shape}')
print(f'Test dataframe shape   :{test_df.shape}')

print(f'Number of train images : {len(train_image_file_paths)}')
print(f'Number of test images  : {len(test_image_file_paths)}')



Train dataframe shape  :(1790594, 17)
Test dataframe shape   :(146853, 3)
Number of train images : 1790594
Number of test images  : 146853


In [8]:
import pandas as pd
import numpy as np
import glob, os

In [9]:
from tensorflow import keras
#import tensor as tf
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Convolution2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.initializers import glorot_uniform
import numpy as np
np.random.seed(1000)

# Initializing the CNN
classifier = Sequential()

# Convolution Step 1
classifier.add(Convolution2D(96, 11, strides = (4, 4), padding = 'valid', input_shape=(224, 224, 3), activation = 'relu'))

# Max Pooling Step 1
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Convolution Step 2
classifier.add(Convolution2D(256, 11, strides = (1, 1), padding='valid', activation = 'relu'))

# Max Pooling Step 2
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='valid'))
classifier.add(BatchNormalization())

# Convolution Step 3
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 4
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 5
classifier.add(Convolution2D(256, 3, strides=(1,1), padding='valid', activation = 'relu'))

# Max Pooling Step 3
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Flattening Step
classifier.add(Flatten())

# Full Connection Step
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 1000, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 38, activation = 'softmax'))
classifier.summary()

2021-09-16 15:58:15.966336: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-16 15:58:15.971378: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-16 15:58:15.971431: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-16 15:58:15.971461: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (230b717e4796): /proc/driver/nvidia/version does not exist
2021-09-16 15:58:15.974083: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other o

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 256)         1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 384)         8

### 

In [10]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
from tensorflow.keras import layers
for i, layer in enumerate(classifier.layers):
   print(i, layer.name)

0 conv2d
1 max_pooling2d
2 batch_normalization
3 conv2d_1
4 max_pooling2d_1
5 batch_normalization_1
6 conv2d_2
7 batch_normalization_2
8 conv2d_3
9 batch_normalization_3
10 conv2d_4
11 max_pooling2d_2
12 batch_normalization_4
13 flatten
14 dense
15 dropout
16 batch_normalization_5
17 dense_1
18 dropout_1
19 batch_normalization_6
20 dense_2
21 dropout_2
22 batch_normalization_7
23 dense_3


In [11]:
# we chose to train the top 2 conv blocks, i.e. we will freeze
# the first 8 layers and unfreeze the rest:
print("Freezed layers:")
for i, layer in enumerate(classifier.layers[:20]):
    print(i, layer.name)
    layer.trainable = False

Freezed layers:
0 conv2d
1 max_pooling2d
2 batch_normalization
3 conv2d_1
4 max_pooling2d_1
5 batch_normalization_1
6 conv2d_2
7 batch_normalization_2
8 conv2d_3
9 batch_normalization_3
10 conv2d_4
11 max_pooling2d_2
12 batch_normalization_4
13 flatten
14 dense
15 dropout
16 batch_normalization_5
17 dense_1
18 dropout_1
19 batch_normalization_6


In [12]:
#trainable parameters decrease after freezing some bottom layers   
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 256)         1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 384)         8

In [13]:
# Compiling the Model
from tensorflow.keras import optimizers
classifier.compile(optimizer=optimizers.SGD(lr=0.001, momentum=0.9, decay=0.005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
# image preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128
PATH = "../input/rsna-str-pulmonary-embolism-detection/"

training_set = train_datagen.flow_from_directory(PATH+'/train',
                                                 target_size=(224, 224),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

testing_set = test_datagen.flow_from_directory(PATH+'/test',
                                            target_size=(224, 224),
                                            batch_size=batch_size,
                                            class_mode='categorical')

Found 0 images belonging to 7279 classes.
Found 0 images belonging to 650 classes.


In [15]:
class_dict = training_set.class_indices
print(class_dict)

{'0003b3d648eb': 0, '000f7f114264': 1, '00102474a2db': 2, '0038fd5f09f5': 3, '0045f113e031': 4, '004a429fc727': 5, '004b06506f54': 6, '004f9a1605e7': 7, '00511e94edec': 8, '005a0dbcb4b7': 9, '005df0f53614': 10, '00617c9fe236': 11, '0062c8f4f1a7': 12, '00681a0d8068': 13, '0072baad76be': 14, '007a7645ced8': 15, '008626c99017': 16, '008c672a586b': 17, '00967023a6d7': 18, '00ba4c616c15': 19, '00c07cd8129d': 20, '00c38669b4fd': 21, '00c4802978cc': 22, '00c56dd61b33': 23, '00c73e5a4e16': 24, '00cf4b2b751b': 25, '00d4f4409f0c': 26, '00db04fdae51': 27, '00f299f37aa8': 28, '010f10503133': 29, '0112fd91fdd0': 30, '011adabde70c': 31, '0121d6824848': 32, '01272322296e': 33, '013358b540bb': 34, '0134eca94635': 35, '0137615ed6ce': 36, '013b3d90394c': 37, '01500f0667e8': 38, '01522eaf958b': 39, '01539f82909c': 40, '015f16e50803': 41, '016074c9499f': 42, '01658812d32b': 43, '0172ef21e2bf': 44, '017320409cc6': 45, '017494a82a0e': 46, '01796cd8f2cd': 47, '017b68b10099': 48, '018b5097a129': 49, '0198019e

In [16]:
li = list(class_dict.keys())
print(li)

['0003b3d648eb', '000f7f114264', '00102474a2db', '0038fd5f09f5', '0045f113e031', '004a429fc727', '004b06506f54', '004f9a1605e7', '00511e94edec', '005a0dbcb4b7', '005df0f53614', '00617c9fe236', '0062c8f4f1a7', '00681a0d8068', '0072baad76be', '007a7645ced8', '008626c99017', '008c672a586b', '00967023a6d7', '00ba4c616c15', '00c07cd8129d', '00c38669b4fd', '00c4802978cc', '00c56dd61b33', '00c73e5a4e16', '00cf4b2b751b', '00d4f4409f0c', '00db04fdae51', '00f299f37aa8', '010f10503133', '0112fd91fdd0', '011adabde70c', '0121d6824848', '01272322296e', '013358b540bb', '0134eca94635', '0137615ed6ce', '013b3d90394c', '01500f0667e8', '01522eaf958b', '01539f82909c', '015f16e50803', '016074c9499f', '01658812d32b', '0172ef21e2bf', '017320409cc6', '017494a82a0e', '01796cd8f2cd', '017b68b10099', '018b5097a129', '0198019eb647', '019b8e02a175', '01ab604d3171', '01b3538d15d6', '01baa1e9a2d6', '01bab1b7694c', '01d03c276180', '01d21d521f2e', '01d5e1117b08', '01d7afb6c23c', '01df4f681d67', '01e0e5538bef', '01f15f

In [17]:
train_num = training_set.samples
test_num = testing_set.samples

In [18]:
#fitting images to CNN
history = classifier.fit_generator(training_set, testing_set,
                         #steps_per_epoch=train_num//batch_size,
                         #validation_data=valid_set,
                         epochs=5, verbose=1, shuffle=True)
                         #validation_steps=valid_num//batch_size,
                         #callbacks=callbacks_list)
score=classifier.evaluate(training_set, testing_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

ValueError: Asked to retrieve element 0, but the Sequence has length 0